In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt



###UP IS ORIGIN

import math
import torch.optim as optim


### SEC 3
from torchvision.utils import save_image






# custom dataloader for .npy file
class numpyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = torch.from_numpy(data).float()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x
    
    def __len__(self):
        return len(self.data)


In [2]:
### Setting up Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Below codes activates when want to use cpu
#device = torch.device('cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce GTX 1050
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
transform       = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                            ])

print("Load traindata..")
# for training

BATCH_SIZE = 10

traindata       = np.load('train.npy')
traindataset    = numpyDataset(traindata, transform)
trainloader     = DataLoader(traindataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print("Train data is ready")


Load traindata..
Train data is ready


In [4]:
size_img = 120 * 80
size_layer_1 = 100
size_layer_2 = 50
size_layer_3 = 12
size_layer_4 = 3

class autoencoder(nn.Module):
    '''
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(size_img, size_layer_1),
            nn.ReLU(True),
            nn.Linear(size_layer_1, size_layer_2),
            nn.ReLU(True), nn.Linear(size_layer_2, size_layer_3), nn.ReLU(True), nn.Linear(size_layer_3, size_layer_4))
        self.decoder = nn.Sequential(
            nn.Linear(size_layer_4, size_layer_3),
            nn.ReLU(True),
            nn.Linear(size_layer_3, size_layer_2),
            nn.ReLU(True),
            nn.Linear(size_layer_2, size_layer_1),
            nn.ReLU(True), nn.Linear(size_layer_1, size_img), nn.Tanh())
    '''
    def __init__(self) :
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential (
            nn.Conv2d(1, 6, kernel_size = 3),
            nn.ReLU(True),
            nn.Conv2d(6,16,kernel_size=5),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16,6,kernel_size=5),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,1,kernel_size=3),
            nn.ReLU(True))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    

    
max_epochs = 60000 # For prevent system down

learning_rate = 1e-5

In [ ]:
model = autoencoder().cuda()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)
criterion = nn.MSELoss()

In [ ]:
epoch_history = []
loss_history = []

for epoch in range(max_epochs):
    # =========== Saving Origin imgs =========== #
    if epoch == 0 :
        for data in enumerate(traindata) :
            img = torch.tensor(data[1]).cuda()
            to_img = transforms.ToPILImage()
            pic_origin = to_img(img.view(120,80).cpu())
            pic_origin.save('./mlp_img/{}_origin.jpg'.format(data[0]))

    # ================= Train ===================
    loss_logs = []
    for data in enumerate(trainloader):
        
        order = data[0]
        img = data[1].clone().detach()
   
        img = img.cuda()
     
        # Create Noisy img
        
        # Random sigma between 0.01 - 0.04
        noise_level = torch.tensor(0.01) + torch.rand(1) * 0.03
        normal_distrib = torch.distributions.Normal(torch.tensor([0.0]),noise_level)
        noise = normal_distrib.sample(img.shape).view(img.shape).cuda()
        
        noisy_img = img + noise
        
        # ===================forward=====================
        output = model(noisy_img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        loss_logs.append(loss.item())
        optimizer.step()
        
        
        # ================= This code for Debug ====================#
        if epoch == 1 :
            if order == 2 :
                to_img = transforms.ToPILImage()
                pic_origin = to_img(img[2].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(21))
                
                pic_origin = to_img(img[6].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(25))
            else :
                to_img = transforms.ToPILImage()
                pic_origin = to_img(img[3].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(42))
            
                
        if order == 2 or order == 4 :
            if order == 2 :
                to_img = transforms.ToPILImage()
                
                ## 21
                pic_input = to_img(noisy_img[2].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(21,epoch))
                 
                pic_output = to_img(output[2].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(21,epoch))

                ## 25
                pic_input = to_img(noisy_img[6].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(25,epoch))
                
                
                pic_output = to_img(output[6].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(25,epoch))
            else :
                ## 42
                to_img = transforms.ToPILImage()
                pic_input = to_img(noisy_img[3].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(42,epoch))
                
                
                pic_output = to_img(output[3].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(42,epoch))
    epoch_loss = np.mean(loss_logs)
    
        
    # ===================log========================
    print('[epoch {}], loss: (training) {:.10f}'
          .format(epoch, epoch_loss))
    
    
    # ================ loss diff check ============
    loss_history.append(epoch_loss)
    epoch_history.append(epoch)
    if epoch != 0 :
        loss_diff = abs(loss_history[epoch] - loss_history[epoch-1])
        if loss_diff <= 0.0000001 :
            exit_flag += 1
            if exit_flag >= 3 :
                print("loss is converged")
                break
        else :
            exit_flag = 0
        

[epoch 0], loss: (training) 0.0380979313
[epoch 1], loss: (training) 0.0193983827
[epoch 2], loss: (training) 0.0155549587
[epoch 3], loss: (training) 0.0144717908
[epoch 4], loss: (training) 0.0134065277
[epoch 5], loss: (training) 0.0122173900
[epoch 6], loss: (training) 0.0108992030
[epoch 7], loss: (training) 0.0089657759
[epoch 8], loss: (training) 0.0067445223
[epoch 9], loss: (training) 0.0057483523
[epoch 10], loss: (training) 0.0053915102
[epoch 11], loss: (training) 0.0050962762
[epoch 12], loss: (training) 0.0048187195
[epoch 13], loss: (training) 0.0045342858
[epoch 14], loss: (training) 0.0042778062
[epoch 15], loss: (training) 0.0040292341
[epoch 16], loss: (training) 0.0037927676
[epoch 17], loss: (training) 0.0035635257
[epoch 18], loss: (training) 0.0033430112
[epoch 19], loss: (training) 0.0031344127
[epoch 20], loss: (training) 0.0029340943
[epoch 21], loss: (training) 0.0027588578
[epoch 22], loss: (training) 0.0025904421
[epoch 23], loss: (training) 0.0024418584
[e

[epoch 194], loss: (training) 0.0005223248
[epoch 195], loss: (training) 0.0005376362
[epoch 196], loss: (training) 0.0005243147
[epoch 197], loss: (training) 0.0005262587
[epoch 198], loss: (training) 0.0005277158
[epoch 199], loss: (training) 0.0005313922
[epoch 200], loss: (training) 0.0005216013
[epoch 201], loss: (training) 0.0005266057
[epoch 202], loss: (training) 0.0005275429
[epoch 203], loss: (training) 0.0005247836
[epoch 204], loss: (training) 0.0005278719
[epoch 205], loss: (training) 0.0005240430
[epoch 206], loss: (training) 0.0005312787
[epoch 207], loss: (training) 0.0005128137
[epoch 208], loss: (training) 0.0005271566
[epoch 209], loss: (training) 0.0005225285
[epoch 210], loss: (training) 0.0005236604
[epoch 211], loss: (training) 0.0005220634
[epoch 212], loss: (training) 0.0005182615
[epoch 213], loss: (training) 0.0005233177
[epoch 214], loss: (training) 0.0005082347
[epoch 215], loss: (training) 0.0005263913
[epoch 216], loss: (training) 0.0005139630
[epoch 217]

[epoch 385], loss: (training) 0.0004813597
[epoch 386], loss: (training) 0.0004986992
[epoch 387], loss: (training) 0.0004755300
[epoch 388], loss: (training) 0.0004725689
[epoch 389], loss: (training) 0.0004740632
[epoch 390], loss: (training) 0.0004763296
[epoch 391], loss: (training) 0.0004837596
[epoch 392], loss: (training) 0.0004816398
[epoch 393], loss: (training) 0.0004749468
[epoch 394], loss: (training) 0.0004804643
[epoch 395], loss: (training) 0.0004741905
[epoch 396], loss: (training) 0.0004830582
[epoch 397], loss: (training) 0.0004752625
[epoch 398], loss: (training) 0.0004716134
[epoch 399], loss: (training) 0.0004849218
[epoch 400], loss: (training) 0.0004826899
[epoch 401], loss: (training) 0.0004833986
[epoch 402], loss: (training) 0.0004851094
[epoch 403], loss: (training) 0.0004840819
[epoch 404], loss: (training) 0.0004617236
[epoch 405], loss: (training) 0.0004808699
[epoch 406], loss: (training) 0.0004748955
[epoch 407], loss: (training) 0.0004760652
[epoch 408]

[epoch 576], loss: (training) 0.0004488590
[epoch 577], loss: (training) 0.0004681431
[epoch 578], loss: (training) 0.0004596660
[epoch 579], loss: (training) 0.0004688451
[epoch 580], loss: (training) 0.0004485371
[epoch 581], loss: (training) 0.0004494889
[epoch 582], loss: (training) 0.0004625922
[epoch 583], loss: (training) 0.0004571358
[epoch 584], loss: (training) 0.0004675810
[epoch 585], loss: (training) 0.0004660360
[epoch 586], loss: (training) 0.0004545948
[epoch 587], loss: (training) 0.0004569477
[epoch 588], loss: (training) 0.0004496857
[epoch 589], loss: (training) 0.0004609179
[epoch 590], loss: (training) 0.0004605003
[epoch 591], loss: (training) 0.0004584709
[epoch 592], loss: (training) 0.0004553615
[epoch 593], loss: (training) 0.0004655775
[epoch 594], loss: (training) 0.0004745505
[epoch 595], loss: (training) 0.0004650819
[epoch 596], loss: (training) 0.0004541687
[epoch 597], loss: (training) 0.0004645678
[epoch 598], loss: (training) 0.0004564648
[epoch 599]

[epoch 767], loss: (training) 0.0004487478
[epoch 768], loss: (training) 0.0004433203
[epoch 769], loss: (training) 0.0004584791
[epoch 770], loss: (training) 0.0004493733
[epoch 771], loss: (training) 0.0004441051
[epoch 772], loss: (training) 0.0004422647
[epoch 773], loss: (training) 0.0004563982
[epoch 774], loss: (training) 0.0004503002
[epoch 775], loss: (training) 0.0004512125
[epoch 776], loss: (training) 0.0004553483
[epoch 777], loss: (training) 0.0004420470
[epoch 778], loss: (training) 0.0004577861
[epoch 779], loss: (training) 0.0004491512
[epoch 780], loss: (training) 0.0004555128
[epoch 781], loss: (training) 0.0004665783
[epoch 782], loss: (training) 0.0004534222
[epoch 783], loss: (training) 0.0004514072
[epoch 784], loss: (training) 0.0004406597
[epoch 785], loss: (training) 0.0004486563
[epoch 786], loss: (training) 0.0004577829
[epoch 787], loss: (training) 0.0004542546
[epoch 788], loss: (training) 0.0004500108
[epoch 789], loss: (training) 0.0004489083
[epoch 790]

...

[epoch 5447], loss: (training) 0.0004078215
[epoch 5448], loss: (training) 0.0004160966
[epoch 5449], loss: (training) 0.0003986978
[epoch 5450], loss: (training) 0.0004035241
[epoch 5451], loss: (training) 0.0004073362
[epoch 5452], loss: (training) 0.0003968191
[epoch 5453], loss: (training) 0.0004081067
[epoch 5454], loss: (training) 0.0004069379
[epoch 5455], loss: (training) 0.0003951471
[epoch 5456], loss: (training) 0.0004230242
[epoch 5457], loss: (training) 0.0004065147
[epoch 5458], loss: (training) 0.0004142951
[epoch 5459], loss: (training) 0.0004111036
[epoch 5460], loss: (training) 0.0004005411
[epoch 5461], loss: (training) 0.0003957463
[epoch 5462], loss: (training) 0.0004058125
[epoch 5463], loss: (training) 0.0004075980
[epoch 5464], loss: (training) 0.0004072399
[epoch 5465], loss: (training) 0.0004226849
[epoch 5466], loss: (training) 0.0004185146
[epoch 5467], loss: (training) 0.0004130743
[epoch 5468], loss: (training) 0.0004138592
[epoch 5469], loss: (training) 0

[epoch 5634], loss: (training) 0.0003987571
[epoch 5635], loss: (training) 0.0004068846
[epoch 5636], loss: (training) 0.0004054441
[epoch 5637], loss: (training) 0.0004051104
[epoch 5638], loss: (training) 0.0004078792
[epoch 5639], loss: (training) 0.0004060390
[epoch 5640], loss: (training) 0.0004095764
[epoch 5641], loss: (training) 0.0004103910
[epoch 5642], loss: (training) 0.0004048712
[epoch 5643], loss: (training) 0.0004112460
[epoch 5644], loss: (training) 0.0004120639
[epoch 5645], loss: (training) 0.0004048238
[epoch 5646], loss: (training) 0.0004138307
[epoch 5647], loss: (training) 0.0004073331
[epoch 5648], loss: (training) 0.0004031889
[epoch 5649], loss: (training) 0.0004223481
[epoch 5650], loss: (training) 0.0004061178
[epoch 5651], loss: (training) 0.0004045365
[epoch 5652], loss: (training) 0.0004066579
[epoch 5653], loss: (training) 0.0004074278
[epoch 5654], loss: (training) 0.0004082448
[epoch 5655], loss: (training) 0.0004080478
[epoch 5656], loss: (training) 0

[epoch 5821], loss: (training) 0.0004168815
[epoch 5822], loss: (training) 0.0003973135
[epoch 5823], loss: (training) 0.0004095561
[epoch 5824], loss: (training) 0.0003976811
[epoch 5825], loss: (training) 0.0003983199
[epoch 5826], loss: (training) 0.0004182534
[epoch 5827], loss: (training) 0.0004114067
[epoch 5828], loss: (training) 0.0004046372
[epoch 5829], loss: (training) 0.0004036666
[epoch 5830], loss: (training) 0.0004087471
[epoch 5831], loss: (training) 0.0004081038
[epoch 5832], loss: (training) 0.0004079799
[epoch 5833], loss: (training) 0.0004107675
[epoch 5834], loss: (training) 0.0004089044
[epoch 5835], loss: (training) 0.0004093540
[epoch 5836], loss: (training) 0.0004086245
[epoch 5837], loss: (training) 0.0004122200
[epoch 5838], loss: (training) 0.0004130303
[epoch 5839], loss: (training) 0.0004153881
[epoch 5840], loss: (training) 0.0004023683
[epoch 5841], loss: (training) 0.0004085396
[epoch 5842], loss: (training) 0.0004111951
[epoch 5843], loss: (training) 0

[epoch 6008], loss: (training) 0.0004084514
[epoch 6009], loss: (training) 0.0004148437
[epoch 6010], loss: (training) 0.0003895804
[epoch 6011], loss: (training) 0.0004002591
[epoch 6012], loss: (training) 0.0004142730
[epoch 6013], loss: (training) 0.0004044888
[epoch 6014], loss: (training) 0.0004013124
[epoch 6015], loss: (training) 0.0004066507
[epoch 6016], loss: (training) 0.0004014294
[epoch 6017], loss: (training) 0.0004150768
[epoch 6018], loss: (training) 0.0003994277
[epoch 6019], loss: (training) 0.0004041213
[epoch 6020], loss: (training) 0.0004043051
[epoch 6021], loss: (training) 0.0004079531
[epoch 6022], loss: (training) 0.0004110028
[epoch 6023], loss: (training) 0.0004027388
[epoch 6024], loss: (training) 0.0004031647
[epoch 6025], loss: (training) 0.0004024645
[epoch 6026], loss: (training) 0.0004174297
[epoch 6027], loss: (training) 0.0004078876
[epoch 6028], loss: (training) 0.0004066742
[epoch 6029], loss: (training) 0.0004106955
[epoch 6030], loss: (training) 0

[epoch 6195], loss: (training) 0.0004090045
[epoch 6196], loss: (training) 0.0003976724
[epoch 6197], loss: (training) 0.0003925552
[epoch 6198], loss: (training) 0.0004081579
[epoch 6199], loss: (training) 0.0004115204
[epoch 6200], loss: (training) 0.0004032598
[epoch 6201], loss: (training) 0.0003998647
[epoch 6202], loss: (training) 0.0004125256
[epoch 6203], loss: (training) 0.0004089731
[epoch 6204], loss: (training) 0.0003958713
[epoch 6205], loss: (training) 0.0004143055
[epoch 6206], loss: (training) 0.0004069344
[epoch 6207], loss: (training) 0.0004166200
[epoch 6208], loss: (training) 0.0004071923
[epoch 6209], loss: (training) 0.0004066462
[epoch 6210], loss: (training) 0.0004027233
[epoch 6211], loss: (training) 0.0004037064
[epoch 6212], loss: (training) 0.0004077044
[epoch 6213], loss: (training) 0.0004061155
[epoch 6214], loss: (training) 0.0004256896
[epoch 6215], loss: (training) 0.0004084975
[epoch 6216], loss: (training) 0.0004067336
[epoch 6217], loss: (training) 0

In [ ]:

stop_idx = 0

for epoch in range(NUM_EPOCH):
    for batch_idx, data in enumerate(trainloader):
        
        #data.shape = (1,1,120,80)
        
        optimizer.zero_grad()
        
        origin_image   = data[0]
        # origin_image.shape = (1,120,80)
        ######## 으으
        origin_image = origin_image.view(1,120,80)
        
        #### section
        noise_image_data = Variable(origin_image) + 0.01
        print(noise_image_data.shape)
        target = Variable(origin_image)
        #target = target.long()
        print(target.shape)
        
        
        output = model(noise_image_data)
        output = output.view(1,120,80)
        print(output.shape)
        print(target.shape)
        loss = objective(output,target)
        
        pred = output.data.max(1)[1]
        print(pred)
        
        
        #loss.backward()
        #optimizer.step()
        
        
        
        '''
        to_img  = transforms.ToPILImage()
        image   = to_img(origin_image)
        
        fig     = plt.figure()
        ax      = fig.add_subplot(1, 1, 1)
        ax.imshow(image, cmap='gray')
        '''
        
        '''
        your code for train
        
        '''
        
        
        
        
        
        #plt.cla()
        #plt.show()
        stop_idx += 1
        if (stop_idx == 30) :
            break
    break


In [ ]:


# for testing
testdata        = np.load('test.npy')
testdataset     = numpyDataset(testdata, transform)
testloader      = DataLoader(testdataset, batch_size=1, shuffle=False, num_workers=2)

result_for_submit = None    # this is for submit file

for batch_idx, data in enumerate(testloader):

    result_of_test = data

    if batch_idx == 0:
        result_for_submit = result_of_test
    else:
        try:
            result_for_submit = torch.cat([result_for_submit, result_of_test], dim=0)

        except RuntimeError:
            transposed = torch.transpose(result_of_test, 2, 3)
            result_for_submit = torch.cat([result_for_submit, transposed], dim=0)

# the submit_file.shape must be (400,1,120,80) 
submit_file = result_for_submit.detach().numpy()
np.save('your_name.npy', submit_file)

